# Hartree Fock Approximation. 

In [ ]:
# Notebook serves as verification of Onsager-Casimir Relation

## Packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Path to modules

In [ ]:
import sys

# sys.path.insert(0, '<path_to_modules>')

## Scattering Region

In [ ]:
import ChiralChainModel_git
import Geometry_Git
Geometry_Git.plot3d_chain(a=1,c=1,M=1,N=8)

## Functions for electron density

In [ ]:
import Integration_Trapz_git

In [ ]:
import negf_git


## Define Scattering Region

In [ ]:
Lm = 8             #number of sites 
chirality = True   #handedness of molecule 
N = Lm             # Number of sites within one winding
M = 1              # Number of windings
a = 1              # radius of helix
c = 1              # pitch of helix


epsilon = 0 # onsite energy
t = 2.4     # hopping paramter

# List of Spin-Orbit Coupling parameter
lambdalist = [(1*(10**-1))*t,(1*(10**-3))*t]
# List of interaction strength
Ulist = [ 0.5*t, 2*t] 


#Gamma Matrix
kmax  = 4      # Number spin up + down sites coupled to each lead
gamma = 0.5    # coupling strength
pz    = 0.5    # magnetic polarization

pz_P = abs(pz)
pz_M = -abs(pz)


T = 300 # Kelvin Temperature of leads
betaL,betaR = negf_git.func_beta(T), negf_git.func_beta(T) # 1/ElectronVolt




# Bias window

In [ ]:
Vmax = 0  # Maximum bias voltage [eV]
V_list_total = [0]

# Energies to integrate over, Fermi Energy

In [ ]:
def func_energies(Hamiltonian0,U,npoints):
    '''
    Input:
    Hamiltonian0 = molecule without interaction (U=0)
    U = interaction strength
    npoints = number of energy points in window [emin,emax]
    Output:
    emax = upper bound of intergral
    emin = lowest eigenvalue of Hamiltonian0
    energiesreal = list of energies between [emin-1000,emax] (make sure the emax < ef +Vmax/2)
    eflist = list of fermi energies for which to calculate electron densities.
    '''
    evlist = np.linalg.eigh(Hamiltonian0)[0]
    e_lumo = evlist[int(Hamiltonian0.shape[0]/2)-1]
    e_homo = evlist[int(Hamiltonian0.shape[0]/2)]
    #Fermi Energy
    hl_gap = e_lumo - e_homo
    
    
    
    #lower,upper bound for Glesser function
    emin = np.round(int(10*min(evlist))/10 - 10,2) #lower bound for integrals
    emax = np.round(int(10*max(evlist))/10 + 10,2)   #lower bound for integrals
    
    
    energies_zero4000 = np.linspace(emin-1000,emin,2000) #In wide band the density integrand have very long "tail"
   
    energiesreal_prime = np.linspace(emin,emax,npoints)
    
    energiesreal = Integration_Trapz_git.jointwolist(energies_zero4000,energiesreal_prime)

#     eflist = [U/2 + abs(np.round( 0  + kappa*hl_gap/2,11)) for kappa in [0.25] ]
    
    #IMPORTANT: Note that ef = 0 is the "symmetric fermi energy", 
    #           since we shifted U<n> -> U(<n>-1/2) in the module: "Intragration_Trapz".
    eflist = [ 0, 1,2  ] 
    
    return emin,emax,npoints,energiesreal,eflist

# Self-Consistent criteria & Integration

Comment on the convergence: Hartree Fock converges well for small U i.e. U<t.

In [ ]:
tol = 10**-5
max_iteration = 100

npoints= 8000

#### Set Paramters

In [ ]:
U       = Ulist[0]
lambda1 = lambdalist[0]
total_list = []

In [ ]:
print(total_list)

In [ ]:

GammaR,GammaLP,GammaLM,Hamiltonian0,hamiltonian_shape= ChiralChainModel_git.system_hamiltonian0(Lm,
                                                                                epsilon,t, 
                                                                          lambda1,chirality,
                                                                          a,c,M,N,
                                                                            kmax,gamma,abs(pz)
                                                                        )
emin,emax,npoints,energiesreal,eflist = func_energies(Hamiltonian0,U,npoints)


for i in range(len(eflist)):
    ef = eflist[i]

    print(U/t, lambda1/t,ef)



    nP_list_total, convglistP = Integration_Trapz_git.self_consistent_trapz(V_list_total,Vmax,
                                                                          max_iteration,ef,
                                                                        U,
                                                                        Hamiltonian0,
                                                                        GammaLP,GammaR, 
                                                                        betaL, betaR,tol,energiesreal)



    nM_list_total, convglistM = Integration_Trapz_git.self_consistent_trapz(V_list_total,Vmax,
                                                                          max_iteration,
                                                                            ef,
                                                                        U,
                                                                        Hamiltonian0,
                                                                        GammaLM,GammaR, 
                                                                        betaL, betaR,tol,energiesreal)



    V_list_convg,nP_list_conv,nM_list_conv    = Integration_Trapz_git.converged_lists(V_list_total,
                                                                              nP_list_total , convglistP,
                                                                                  nM_list_total, convglistM)


    total_list.append([V_list_convg,nP_list_conv,nM_list_conv])

# Onsager Casimir

The transmissions satisfies $T_{LR}(m,V=0) = T_{LR}(-m,V=0)$, thus  Onsager-Casimir is satisfied.

In [ ]:
energies = np.linspace(-10,10,200)

In [ ]:
for i in range(len(eflist)):
    ef = eflist[i]
    V_list_convg,nP_list_conv,nM_list_conv = total_list[i]
    HP = negf_git.Hamiltonian_HF(nP_list_conv[0],U,Hamiltonian0)
    HM = negf_git.Hamiltonian_HF(nM_list_conv[0],U,Hamiltonian0)
    
    deltaT = [ negf_git.TLR(energy,HP ,GammaLP,GammaR )-negf_git.TLR(energy,HM ,GammaLM,GammaR ) for energy in energies]
    
    plt.title('$T_{LR}(m,V=0)- T_{LR}(-m,V=0)$')
    plt.plot(energies,deltaT,label = '$E_F = {}$'.format(ef))
    plt.xlabel('Energy [eV]')
    plt.ylabel('Transsmssion')
    plt.legend()
    plt.show()